# Reciprocal lattice and k-point sampling

In [ ]:
import numpy as np
import plotly.graph_objects as go

from pymatgen.core import Lattice, Structure
from pymatgen_plotly import Figure


In [ ]:
def fcc_conv(element: str, a: float):
    # lattice=Lattice.cubic(a)
    return Structure(
        lattice=Lattice([[a, 0, 0], [0, a, 0], [0, 0, a]]),
        species=4 * [element],
        coords=[[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]],
    )

def fcc_prim(element: str, a: float):
    # a_prim = a * np.sqrt(2) / 2
    # lattice=Lattice.rhombohedral(a_prim, 60)
    return Structure(
        lattice=Lattice([[a/2, a/2, 0], [0, a/2, a/2], [a/2, 0, a/2]]),
        species=[element],
        coords=[[0, 0, 0]],
    )

def fcc_reduced(element: str, a: float):
    return Structure(
        lattice=Lattice([[a/2, -a/2, 0], [a/2, a/2, 0], [0, 0, a]]),
        species=2 * [element],
        coords=[[0, 0, 0], [.5, .5, .5]]
    )

def build_grid(n, shift=None):
    r = np.linspace(0, 1, n, endpoint=False)
    grid = np.array([[x,y,z] for x in r for y in r for z in r])

    if shift:
        offset = 1/n * np.array(shift)
        return grid + offset

    return grid



In [ ]:
struct_conv = fcc_conv('Al', 4.)
struct_prim = fcc_prim('Al', 4.)
struct_reduced = fcc_reduced('Al', 4.)

In [ ]:
facets = struct_prim.lattice.get_brillouin_zone()
# facets

In [ ]:
# brillouin_zone

fig = Figure()
fig._add_unitcell(struct_prim.lattice.reciprocal_lattice.matrix)


facets = struct_prim.lattice.get_brillouin_zone()
traces = []
for facet in facets:
    x, y, z = np.stack(facet).T
    trace = go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50)
    # trace = go.Mesh3d(x=x, y=y, z=z, dsfasd=2)
    traces.append(trace)


fig.fig.add_traces(traces)
fig.fig.update_scenes(camera_projection_type="orthographic")

fig.show()

In [ ]:
# k-point grid

fig = Figure()
fig._add_unitcell(struct_conv.lattice.reciprocal_lattice.matrix)
fig._add_unitcell(struct_reduced.lattice.reciprocal_lattice.matrix)

grid = struct_conv.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(4, shift=[0.5, 0.5, 0.5]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)



grid = struct_reduced.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(4, shift=[0.5, 0.0, 0.5]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)

grid = struct_reduced.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(4, shift=[0.0, 0.5, 0.5]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)

fig.fig.update_scenes(camera_projection_type="orthographic")

fig.show()

In [ ]:
# atom-centered k-point grid

fig = Figure()
fig._add_unitcell(struct_conv.lattice.reciprocal_lattice.matrix)
fig._add_unitcell(struct_reduced.lattice.reciprocal_lattice.matrix)

grid = struct_conv.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(4))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)



grid = struct_reduced.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(4, shift=[0., 0., 0.]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)

grid = struct_reduced.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(4, shift=[.5, .5, 0.]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)

fig.fig.update_scenes(camera_projection_type="orthographic")

fig.show()

In [ ]:
# atom-centered

fig = Figure()
fig._add_unitcell(struct_prim.lattice.reciprocal_lattice.matrix)
fig._add_unitcell(struct_reduced.lattice.reciprocal_lattice.matrix)

n = 2
grid = struct_prim.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(n, shift=[.0, .0, .0]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)

grid = struct_prim.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(n, shift=[.5, .5, 0.]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)

grid = struct_prim.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(n, shift=[0., .5, .5]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)

grid = struct_prim.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(n, shift=[.5, 0., .5]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers')
fig.fig.add_trace(trace)


grid = struct_reduced.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(n, shift=[0., 0., 0.]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers', marker=dict(size=4,color='blue'))
fig.fig.add_trace(trace)

grid = struct_reduced.lattice.reciprocal_lattice.get_cartesian_coords(build_grid(n, shift=[.5, .5, 0.]))
x,y,z = grid.T
trace = go.Scatter3d(x=x,y=y,z=z, mode='markers', marker=dict(size=4,color='blue'))
fig.fig.add_trace(trace)

fig.fig.update_scenes(camera_projection_type="orthographic")
fig.show()